### Código de Exploración de Datos Multivariados de los estudiantes Milessi Ayrton y Torres Santiago

### Cargamos las librerias a utilizar, la base de datos y funciones a utilizar

In [ ]:
# -*- coding: utf-8 -*-

In [ ]:
library(dplyr)
library(readxl) 
library(stringr)
library(writexl)
library(ggplot2)

In [ ]:
# Función para sacar la moda
Mode <- function(x) {
  ux <- unique(x)
  ux[which.max(tabulate(match(x, ux)))]
}

In [ ]:
# Función para evaluar correlación
evaluar_correlacion <- function(spe) {
  if (spe >= 0.8) {
    return("Correlación muy alta")
  } else if (spe >= 0.6) {
    return("Correlación alta")
  } else if (spe >= 0.4) {
    return("Correlación moderada")
  } else if (spe >= 0.2) {
    return("Correlación baja")
  } else {
    return("No hay correlación")
  }
}

In [ ]:
Victoria <- read.csv("C:/Users/ayrto/OneDrive/Escritorio/Exploración de Datos Multivariados/1 - Trabajo Practico/VICTORIA.csv") 
dim(Victoria) #Mostramos las dimensiones de la base de datos original

### Exploramos la base de datos

In [ ]:
names(Victoria)

In [ ]:
# Cambiamos los nombres de las variables para un mejor entendimiento
Victoria <- rename(Victoria, c("CodigoUnicoEscolar" = "CUE", "ModEnseñanza" = "ModEnsenaza", "año" = "anioLectivo", "esMultiaño" ="esMultianio", "IdModEnseñanza" = "IdModEnsenanza","NivelEnseñanza" = "NivelEnsenanza"))

In [ ]:
# Descubrimos que al intentar modificar el nombre de una varible, esta se encuentra dos veces en el conjunto de datos
Victoria$ddivision <- NULL #Entonces procedemos a eliminar una de las variables duplicada
names(Victoria) #Verificamos si se elimino correctamente

In [ ]:
# Mostramos las variables del dataset, nos ayuda a identificar el tipo de cada variable
str(Victoria)

In [ ]:
# Reemplazamos las notas "EPAC", "UP" y "PA" 
Victoria$nota[Victoria$nota %in% c("EPAC")] <- 5
Victoria$nota[Victoria$nota %in% c("PA")] <- 6
Victoria$nota[Victoria$nota %in% c("UP")] <- 7
Victoria$nota <- suppressWarnings(as.numeric(Victoria$nota)) #Convertimos la variable nota a tipo numérico

In [ ]:
# Ahora dividiremos el dataset en dos: Primaria y Secundaria
# Nueva base de datos para primaria
Victoria_primaria <- filter(Victoria, NivelEnseñanza == "PRIMARIO" & ( ModEnseñanza == "Comun"| ModEnseñanza == "Escuelas NINA") & Modalidad == "Primaria" & esMultiaño == "N") #se guardan los alumnos que cursan la primaria
Victoria_secundaria <- filter(Victoria, NivelEnseñanza == "SECUNDARIO" & (ModEnseñanza == "Comun" | ModEnseñanza == "Técnica" | ModEnseñanza =="Agrotécnica") & esMultiaño == "N" & Modalidad == "Secundaria") #se guardan los alumnos que cursan la secundaria

In [ ]:
length(unique(Victoria_primaria$CodigoUnicoEscolar))   #cantidad de escuelas primarias en Victoria
# Dimensiones de las nuevas bases de datos 
dim(Victoria_primaria)

### Ahora nos enfocaremos en la limpieza de la base de datos de primaria

In [ ]:
# Creamos nuevos data frames para almacenar las notas de cada trimestre
PrimerT_primaria <- filter(Victoria_primaria, periodoEvaluatorio == "Primer Trimestre")
PrimerT_primaria <- rename(PrimerT_primaria,NotasPrimerT = nota)
SegundoT_primaria <- filter(Victoria_primaria, periodoEvaluatorio == "Segundo Trimestre")
SegundoT_primaria <- rename(SegundoT_primaria,NotasSegundoT = nota)
TercerT_primaria <- filter(Victoria_primaria, periodoEvaluatorio == "Tercer Trimestre")
TercerT_primaria <- rename(TercerT_primaria,NotasTercerT = nota)

In [ ]:
# A traves del merge unimos las notas de cada trimestre segun el Documento de cada estudiante y la asignatura
aux <- merge(x = PrimerT_primaria, y = SegundoT_primaria, by = c("Documento", "asignatura"))
auxiliar <- merge(x = aux, y = TercerT_primaria, by = c("Documento", "asignatura"))

In [ ]:
Notas_Victoria_primaria <- select (auxiliar, 'CodigoUnicoEscolar', 'departamento', 'localidad', 'EsPrivada', 'turno' , 'curso' , 'periodoEvaluatorio' , 'Documento', 'asignatura','idalumno', 'observaciones', 'idSubOrganizacion', 'idDivision', 'NivelEnseñanza', 'orden','Nivel', 'esMultiaño', 'IdModEnseñanza', 'Modalidad', 'ModEnseñanza', 'año','NotasPrimerT','NotasSegundoT', 'NotasTercerT')

In [ ]:
# Calculamos el promedio para cada estudiante en las asignaturas que cursa
Notas_Victoria_primaria$Promedio <-round ((as.numeric(Notas_Victoria_primaria$NotasPrimerT)+as.numeric(Notas_Victoria_primaria$NotasSegundoT)+as.numeric(Notas_Victoria_primaria$NotasTercerT))/3,2)

In [ ]:
# Según el promedio conseguido tendra aprobado el año lectivo o tendrá que rendir. 
# Los alumnos que tengas una nota menor a 6 en el tercer trimestre tendrán que rendir.
# Los estudiantes que no tengan las 3 notas correspondientes a cada trimestre tendrán un registro incompleto
Notas_Victoria_primaria <- Notas_Victoria_primaria %>% 
  mutate(Condición = case_when(
    Promedio >= 6 & NotasTercerT >= 6 ~ "Aprobado", 
    Promedio < 6 | NotasTercerT < 6 ~ "Rinde",      
    TRUE ~ "sin registro completo"                    
  ))

In [ ]:
# Pasamos todos los nombres de las asignaturas a mayusculas para una mayor facilidad a la hora de cambiarlos
Notas_Victoria_primaria$asignatura <- toupper(Notas_Victoria_primaria$asignatura)

In [ ]:
# Ponemos un nombre común a las asignaturas que dictan los mismos contenidos pero tienen un nombre diferente
Notas_Victoria_primaria$asignatura <- gsub("MATEMÁTICA|ACOMPAÑAMIENTO AL ESTUDIO MATEMÁTICA", "MATEMATICA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("LITERATURA|ACOMPAÑAMIENTO AL ESTUDIO LENGUA", "LENGUA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("ACOMPAÑAMIENTO AL ESTUDIO SOCIALES", "CIENCIAS SOCIALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("ACOMPAÑAMIENTO AL ESTUDIO NATURALES|TALLER VIDA EN LA NATURALEZA", "CIENCIAS NATURALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE ARTES VISUALES|TALLER DE LENGUAJE DE LAS ARTES VISUALES", "ARTES VISUALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE DEPORTE|DEPORTE", "EDUCACION FISICA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("E.D.I.", "EDI", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE MUSICA|TALLER DE FOLKLORE", "EDUCACION MUSICAL", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE TIC|TALLER DE TIC Y RECREACIÓN DE LA IMAGEN|TALLER TIC|TICS", "TIC", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("IDIOMA EXTRANJERO|INGLÉS|TALLER DE INGLES|TALLER DE INGLÉS|TALLER DE LENGUA EXTRANJERA|INGLEs|TALLER INGLES|TALLER LENGUA EXTRANJERA (INGLES)", "INGLES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE TEATRO|TALLER TEATRO", "TALLER  DE TEATRO", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("INFORMÁTICA", "INFORMATICA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER LABORATORIO", "TALLER DE LABORATORIO", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("AC.EN LENGUA Y CS. SOCIALES|T/A ESTUDIO (LENGUA-CS SOCIALES)|TALLER ACOMPAÑAMIENTO LENGUA Y CIENCIAS SOCIALES", "TALLER DE CS. SOCIALES Y LENGUA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("AC.EN MATEMÁTICA Y CS. NATURALES|T/A ESTUDIO (MATEMATICA-CS NATURALES)|TALLER ACOMPAÑAMIENTO  MATEMÁTICA Y CS. NATURALES", "TALLER DE CS. NATURALES Y MATEMATICA", Notas_Victoria_primaria$asignatura)

In [ ]:
# Verificamos que no se hayan modificado bien los nombres de las asignaturas
table(Notas_Victoria_primaria$asignatura)

In [ ]:
# No todas las asignaturas fueron modificadas correctamente, se procede a realizar denuevo el procedimiento con aquellas que falta cambiar
Notas_Victoria_primaria$asignatura <- str_replace_all(Notas_Victoria_primaria$asignatura, ".*INGLES*.", "INGLES")
Notas_Victoria_primaria$asignatura <- gsub("TALLER ACOMPAÑAMIENTO  MATEMATICA Y CS. NATURALES|T/A ESTUDIO (MATEMATICA-CS NATURALES)|AC.EN MATEMATICA Y CS. NATURALES", "TALLER DE MATEMATICA Y CS. NATURALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- str_replace_all(Notas_Victoria_primaria$asignatura, "T/A ESTUDIO \\(LENGUA-CS SOCIALES\\)", "TALLER DE CS. SOCIALES Y LENGUA")
Notas_Victoria_primaria$asignatura <- str_replace_all(Notas_Victoria_primaria$asignatura, "T/A ESTUDIO \\(MATEMATICA-CS NATURALES\\)", "TALLER DE MATEMATICA Y CS. NATURALES")
# Verificamos nuevamente. Ya tenemos todos los nombres de las materias correctamente
table(Notas_Victoria_primaria$asignatura)

In [ ]:
# Eliminamos un curso multiaño que supero el filtro implementado
Notas_Victoria_primaria <- Notas_Victoria_primaria[Notas_Victoria_primaria$curso != "1-6 - A Prim", ]
table(Notas_Victoria_primaria$curso)
# Ya nos quedamos con los cursos que no son multiaños 

In [ ]:
# Eliminamos aquellos estudiantes que no tienen el promedio completo
Notas_Victoria_primaria <- Notas_Victoria_primaria[Notas_Victoria_primaria$Condición != "sin registro completo", ]

In [ ]:
# Cambiamos a tipo caracter los nombres de los cursos para luego quedarnos con el primer caracter que aparece
# (en cada dato, el primer caracter es el numero del grado
Notas_Victoria_primaria$curso <- as.character(Notas_Victoria_primaria$curso)
Notas_Victoria_primaria$Año_curso <- str_sub(Notas_Victoria_primaria$curso,1, 1)
# Mostramos la cantidad de alumnos en cada grado
table(Notas_Victoria_primaria$Año_curso)

In [ ]:
# Modificamos la variable 'Año_curso' del número al nombre del grado
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "1" , "PRIMERO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "2" , "SEGUNDO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "3" , "TERCERO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "4" , "CUARTO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "5" , "QUINTO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "6" , "SEXTO")

In [ ]:
Notas_Victoria_primaria <- dplyr::select(Notas_Victoria_primaria,'CodigoUnicoEscolar','departamento','EsPrivada','turno','Año_curso','Documento','asignatura','idalumno','NivelEnseñanza','esMultiaño','Modalidad','ModEnseñanza','NotasPrimerT','NotasSegundoT','NotasTercerT','Promedio','Condición')

### Segmentamos la base de datos primaria en seis nuevas bases de datos nuevas. Cada una le pertencerá a cada grado

In [ ]:
Notas_Victoria_1Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "PRIMERO"))
tabla1 <- Notas_Victoria_1Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
print("Alumnos por materia en 1er grado")
tabla1

In [ ]:
Notas_Victoria_2Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "SEGUNDO"))
tabla2 <- Notas_Victoria_2Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
print("Alumnos por materia en 2do grado")
tabla2

In [ ]:
Notas_Victoria_3Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "TERCERO"))
tabla3 <- Notas_Victoria_3Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
print("Alumnos por materia en 3er grado")
tabla3

In [ ]:
Notas_Victoria_4Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "CUARTO"))
tabla4 <- Notas_Victoria_4Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
print("Alumnos por materia en 4to grado")
tabla4

In [ ]:
Notas_Victoria_5Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "QUINTO"))
tabla5 <- Notas_Victoria_5Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
print("Alumnos por materia en 5to grado")
tabla5

In [ ]:
Notas_Victoria_6Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "SEXTO"))
tabla6 <- Notas_Victoria_6Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
print("Alumnos por materia en 6to grado")
tabla6

## Empezaremos con el analisis de la bases de datos de los grados de primaria

*Primero vamos a ir haciendo distintos analisis que describiran el estado de la base de datos en primaria*

In [ ]:
# Estas librerias se ejecuntan en esta parte porque corrompen el codigo
library(ppcor)
library(corrplot)
library(tidyr)
library(fdth)
library("moments")

In [ ]:
print("Tabla de frecuencias de turnos en el nivel primario")
Fa <- table(Notas_Victoria_primaria$turno)
Fr <- prop.table(Fa)
fip <- prop.table(Fa)*100
FA <- cumsum(Fa)
FR <- cumsum(Fr)
tabla_frecuencias_turno <- round(cbind(Fa, Fr, fip,FA, FR),2)
tabla_frecuencias_turno

# Gráfico de sectores de los turnos
pie_commute <- round(100* table(Notas_Victoria_primaria$turno) / length(Notas_Victoria_primaria$turno))
colors = c('#ffeb3b', '#ff6e40','#ed2f52')
pie(pie_commute, labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
    col = colors,
    radius = .90,
    main = "Distribucion Porcentual de la Matricula de 2023
    en el departamento Victoria en cada 
    turno en el nivel primario",
    col.main = "black",
    border = 'black')

In [ ]:
print("Tabla de frecuencias de tipo de gestión en el nivel primario")
Fa <- table(Notas_Victoria_primaria$EsPrivada)
Fr <- prop.table(Fa)
fip <- prop.table(Fa)*100
FA <- cumsum(Fa)
FR <- cumsum(Fr)
tabla_frecuencias_sector <- round(cbind(Fa, Fr, fip,FA, FR),2)
tabla_frecuencias_sector

# Gráfico de sectores de los tipos de gestion
pie_commute <- round(100* table(Notas_Victoria_primaria$EsPrivada) / length(Notas_Victoria_primaria$EsPrivada))
colors = c('#2196f3', '#8bc34a')
etiquetas <- c("Estatal", "Privada")
pie(pie_commute, labels = paste0(etiquetas,'\n', pie_commute, ' %'),
    col = colors,
    radius = .90,
    main = "Distribucion Porcentual de la Matricula de 2023
    en el departamento Victoria por tipo 
    de gestión en el nivel primario",
    col.main = "black",
    border = 'black')

In [ ]:
print("Tabla de frecuencias de la condición final en el nivel primario")
Fa <- table(Notas_Victoria_primaria$Condición)
Fr <- prop.table(Fa)
fip <- prop.table(Fa)*100
FA <- cumsum(Fa)
FR <- cumsum(Fr)
tabla_frecuencias_condicion <- round(cbind(Fa, Fr, fip,FA, FR),2)
tabla_frecuencias_condicion

# Gráfico de sectores de las condiciones finales
pie_commute <- round(100* table(Notas_Victoria_primaria$Condición) / length(Notas_Victoria_primaria$Condición))
colors <- c('#3333FF', '#FF2B00')
pie(pie_commute, labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
    col = colors,
    radius = .90,
    main = "Distribucion Porcentual de la Matricula de 2023 en
    el departamento Victoria segun la condición
    final en el nivel primario",
    col.main = "black",
    border = 'black')

In [ ]:
aux <- distinct(Notas_Victoria_primaria, idalumno, .keep_all = TRUE)
print("Tabla de frecuencias de los matriculados por grado en el nivel primario")
Fa <- table(aux$Año_curso)
Fr <- prop.table(Fa)
fip <- prop.table(Fa)*100
FA <- cumsum(Fa)
FR <- cumsum(Fr)
tabla_frecuencias_cursos <- round(cbind(Fa, Fr, fip,FA, FR),2)
tabla_frecuencias_cursos

# Gráfico de barras de las frecuencias absolutas por grado
datos <- table(aux$Año_curso)
tabla_años <- as.data.frame(datos)
names(tabla_años) <- c("Año", "Freq")

tabla_años <- tabla_años %>%
  arrange(factor(Año, levels = c('PRIMERO', 'SEGUNDO', 'TERCERO', 'CUARTO', 'QUINTO', 'SEXTO')))
barplot(tabla_años$Freq, 
        names.arg = tabla_años$Año, 
        main = "Frecuencias absolutas de estudiantes por grado en primaria",
        col = rainbow(6), 
        ylim = c(0, 600), 
        las = 1, 
        cex.names = 0.9)
text(x = 1:length(tabla_años$Año), 
     y = tabla_años$Freq + 40,
     labels = tabla_años$Freq, 
     pos = 1,
     cex = 1,
     col = "black")

## Análisis de primer grado

In [ ]:
print("Medidas estadisticas descriptivas de primer grado")
Notas_Victoria_1Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>% 
  group_by(asignatura) %>%
  summarise(Conteo = n(),
            Media = round(mean(Promedio, na.rm = TRUE), 2),
            Mediana = round(median(Promedio, na.rm = TRUE), 2),
            Moda = Mode(Promedio),            
            Varianza = round(var(Promedio, na.rm = TRUE), 2),
            Desvio = round(sd(Promedio, na.rm = TRUE), 2),
            Minimo = round(min(Promedio, na.rm = TRUE), 2),
            Maximo = round(max(Promedio, na.rm = TRUE), 2),
            Rango = Maximo - Minimo,
            IQR = round(IQR(Promedio, na.rm = TRUE), 2),
            Q1 = round(quantile(Promedio, 0.25, na.rm = TRUE), 2),
            Q3 = round(quantile(Promedio, 0.75, na.rm = TRUE), 2),
            Sup. = round((Q3 + 1.5 * IQR), 2),
            Inf. = round((Q1 - 1.5 * IQR), 2),
            CoefVar = round(sd(Promedio, na.rm = TRUE) / mean(Promedio, na.rm = TRUE) * 100, 2))

In [ ]:
# Gráfico de boxplot comparativo
datos_filtrados <- Notas_Victoria_1Grado %>%
  filter(asignatura %in% c("CIENCIAS NATURALES", "CIENCIAS SOCIALES", "MATEMATICA", "LENGUA"))

p <- ggplot(datos_filtrados, aes(x = asignatura, y = Promedio, fill = asignatura)) +
  geom_boxplot() +
  ggtitle("                      Comparación de Promedios en materias de primer grado") +  
  labs(
    x = "Materias",
    y = "Promedios"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )
print(p)

In [ ]:
# Histograma de promedios en matemática
notas_mat_primero <- filter(Notas_Victoria_1Grado,asignatura == "MATEMATICA")$Promedio
histograma_mat <- fdt(notas_mat_primero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_mat,
     type= "fh", 
     col= "#FF9932",
     main= "Histograma de los promedios de primer grado
     en la materia Matemática")

# Medidas de distribución
coef_sim = skewness(notas_mat_primero, na.rm=TRUE)
kurtosis = kurtosis(notas_mat_primero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_mat_primero)

In [ ]:
# Histograma de promedios en lengua
notas_len_primero <- filter(Notas_Victoria_1Grado,asignatura == "LENGUA")$Promedio
histograma_len <- fdt(notas_len_primero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_len,
     type= "fh", 
     col= "#FF86FF",
     main= "Histograma de los promedios de primer grado
     en la materia Lengua")

# Medidas de distribución
coef_sim = skewness(notas_len_primero, na.rm=TRUE)
kurtosis = kurtosis(notas_len_primero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_len_primero)

In [ ]:
# Histograma de promedios en ciencias naturales
notas_nat_primero <- filter(Notas_Victoria_1Grado,asignatura == "CIENCIAS NATURALES")$Promedio
histograma_nat <- fdt(notas_nat_primero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_nat,
     type= "fh", 
     col= "#00BB00",
     main= "Histograma de los promedios de primer grado
     en la materia Ciencias Naturales")

# Medidas de distribución
coef_sim = skewness(notas_nat_primero, na.rm=TRUE)
kurtosis = kurtosis(notas_nat_primero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_nat_primero)

In [ ]:
# Histograma de promedios en ciencias sociales
notas_soc_primero <- filter(Notas_Victoria_1Grado,asignatura == "CIENCIAS SOCIALES")$Promedio
histograma_soc <- fdt(notas_soc_primero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_soc,
     type= "fh", 
     col= "#3D87FF",
     main= "Histograma de los promedios de primer grado
     en la materia Ciencia Sociales")

# Medidas de distribución
coef_sim = skewness(notas_soc_primero, na.rm=TRUE)
kurtosis = kurtosis(notas_soc_primero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Tests de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_soc_primero)

In [ ]:
# Unimos en un data frame las materias seleccionadas para calcular correlaciones
df <- suppressWarnings(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "CIENCIAS NATURALES"),"Documento","Promedio") %>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "CIENCIAS SOCIALES"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "MATEMATICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "LENGUA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "EDUCACION FISICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "EDUCACION MUSICAL"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "EDUCACION TECNOLOGICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "ARTES VISUALES"),"Documento","Promedio"),by="Documento"))
df <- rename(df,Prom_Nat=Promedio.x, Prom_Soc=Promedio.y, Prom_Mat=Promedio.x.x, Prom_Len=Promedio.y.y, Prom_EdFisica=Promedio.x.x.x, Prom_Mus=Promedio.y.y.y, Prom_Tecno=Promedio.x.x.x.x, Prom_Art=Promedio.y.y.y.y)

In [ ]:
# Gráfico de la matriz de correlación
matriz_correlacion <- cor(df[, -1], method="spearman")
corrplot(matriz_correlacion, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Matriz de correlación", cex.main = 1.5)

# Gráfico de correlación parcial
variables <- c("Prom_Nat", "Prom_Soc", "Prom_Mat", "Prom_Len")
dataM <- df[, variables]
correlacion_parcial <- pcor(dataM, method = "pearson")$estimate
corrplot(correlacion_parcial, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Correlación parcial", cex.main = 1.5)

## Análisis de Segundo grado

In [ ]:
print("Medidas estadisticas descriptivas de segundo grado")
Notas_Victoria_2Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>% 
  group_by(asignatura) %>%
  summarise(Conteo = n(),
            Media = round(mean(Promedio, na.rm = TRUE), 2),
            Mediana = round(median(Promedio, na.rm = TRUE), 2),
            Moda = Mode(Promedio),            
            Varianza = round(var(Promedio, na.rm = TRUE), 2),
            Desvio = round(sd(Promedio, na.rm = TRUE), 2),
            Minimo = round(min(Promedio, na.rm = TRUE), 2),
            Maximo = round(max(Promedio, na.rm = TRUE), 2),
            Rango = Maximo - Minimo,
            IQR = round(IQR(Promedio, na.rm = TRUE), 2),
            Q1 = round(quantile(Promedio, 0.25, na.rm = TRUE), 2),
            Q3 = round(quantile(Promedio, 0.75, na.rm = TRUE), 2),
            Sup. = round((Q3 + 1.5 * IQR), 2),
            Inf. = round((Q1 - 1.5 * IQR), 2),
            CoefVar = round(sd(Promedio, na.rm = TRUE) / mean(Promedio, na.rm = TRUE) * 100, 2))

In [ ]:
# Gráfico de boxplot comparativo
datos_filtrados <- Notas_Victoria_2Grado %>%
  filter(asignatura %in% c("CIENCIAS NATURALES", "CIENCIAS SOCIALES", "MATEMATICA", "LENGUA"))

p <- ggplot(datos_filtrados, aes(x = asignatura, y = Promedio, fill = asignatura)) +
  geom_boxplot() +
  ggtitle("                      Comparación de Promedios en materias de segundo grado") +  
  labs(
    x = "Materias",
    y = "Promedios"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )
print(p)

In [ ]:
# Histograma de promedios en matemática
notas_mat_segundo <- filter(Notas_Victoria_2Grado,asignatura == "MATEMATICA")$Promedio
histograma_mat <- fdt(notas_mat_segundo, breaks= "Sturges", na.rm= TRUE)
plot(histograma_mat,
     type= "fh", 
     col= "#FF9932",
     main= "Histograma de los promedios de segundo grado
     en la materia Matemática")

# Medidas de distribución
coef_sim = skewness(notas_mat_segundo, na.rm=TRUE)
kurtosis = kurtosis(notas_mat_segundo, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_mat_segundo)

In [ ]:
# Histograma de promedios en lengua
notas_len_segundo <- filter(Notas_Victoria_2Grado,asignatura == "LENGUA")$Promedio
histograma_len <- fdt(notas_len_segundo, breaks= "Sturges", na.rm= TRUE)
plot(histograma_len,
     type= "fh", 
     col= "#FF86FF",
     main= "Histograma de los promedios de segundo grado
     en la materia Lengua")

# Medidas de distribución
coef_sim = skewness(notas_len_segundo, na.rm=TRUE)
kurtosis = kurtosis(notas_len_segundo, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_len_segundo)

In [ ]:
# Histograma de promedios en ciencias naturales
notas_nat_segundo <- filter(Notas_Victoria_2Grado,asignatura == "CIENCIAS NATURALES")$Promedio
histograma_nat <- fdt(notas_nat_segundo, breaks= "Sturges", na.rm= TRUE)
plot(histograma_nat,
     type= "fh", 
     col= "#00BB00",
     main= "Histograma de los promedios de segundo grado
     en la materia Ciencias Naturales")

# Medidas de distribución
coef_sim = skewness(notas_nat_segundo, na.rm=TRUE)
kurtosis = kurtosis(notas_nat_segundo, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_nat_segundo)

In [ ]:
# Histograma de promedios en ciencias sociales
notas_soc_segundo <- filter(Notas_Victoria_2Grado,asignatura == "CIENCIAS SOCIALES")$Promedio
histograma_soc <- fdt(notas_soc_segundo, breaks= "Sturges", na.rm= TRUE)
plot(histograma_soc,
     type= "fh", 
     col= "#3D87FF",
     main= "Histograma de los promedios de segundo grado
     en la materia Ciencias Sociales")

# Medidas de distribución
coef_sim = skewness(notas_soc_segundo, na.rm=TRUE)
kurtosis = kurtosis(notas_soc_segundo, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_soc_segundo)

In [ ]:
# Unimos en un data frame las materias seleccionadas para calcular correlaciones
df <- suppressWarnings(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "CIENCIAS NATURALES"),"Documento","Promedio") %>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "CIENCIAS SOCIALES"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "MATEMATICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "LENGUA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "EDUCACION FISICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "EDUCACION MUSICAL"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "EDUCACION TECNOLOGICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "ARTES VISUALES"),"Documento","Promedio"),by="Documento"))
df <- rename(df,Prom_Nat=Promedio.x, Prom_Soc=Promedio.y, Prom_Mat=Promedio.x.x, Prom_Len=Promedio.y.y, Prom_EdFisica=Promedio.x.x.x, Prom_Mus=Promedio.y.y.y, Prom_Tecno=Promedio.x.x.x.x, Prom_Art=Promedio.y.y.y.y)

In [ ]:
# Gráfico de la matriz de correlación
matriz_correlacion <- cor(df[, -1], method="spearman")
corrplot(matriz_correlacion, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Matriz de correlación", cex.main = 1.5)

# Gráfico de correlación parcial
variables <- c("Prom_Nat", "Prom_Soc","Prom_Mat", "Prom_Len", "Prom_Mus", "Prom_Tecno")
dataM <- df[, variables]
correlacion_parcial <- pcor(dataM, method = "pearson")$estimate
corrplot(correlacion_parcial, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Correlación parcial", cex.main = 1.5)

## Análisis de Tercer grado

In [ ]:
print("Medidas estadisticas descriptivas de tercer grado")
Notas_Victoria_3Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>% 
  group_by(asignatura) %>%
  summarise(Conteo = n(),
            Media = round(mean(Promedio, na.rm = TRUE), 2),
            Mediana = round(median(Promedio, na.rm = TRUE), 2),
            Moda = Mode(Promedio),            
            Varianza = round(var(Promedio, na.rm = TRUE), 2),
            Desvio = round(sd(Promedio, na.rm = TRUE), 2),
            Minimo = round(min(Promedio, na.rm = TRUE), 2),
            Maximo = round(max(Promedio, na.rm = TRUE), 2),
            Rango = Maximo - Minimo,
            IQR = round(IQR(Promedio, na.rm = TRUE), 2),
            Q1 = round(quantile(Promedio, 0.25, na.rm = TRUE), 2),
            Q3 = round(quantile(Promedio, 0.75, na.rm = TRUE), 2),
            Sup. = round((Q3 + 1.5 * IQR), 2),
            Inf. = round((Q1 - 1.5 * IQR), 2),
            CoefVar = round(sd(Promedio, na.rm = TRUE) / mean(Promedio, na.rm = TRUE) * 100, 2))

In [ ]:
# Gráfico de boxplot comparativo
datos_filtrados <- Notas_Victoria_3Grado %>%
  filter(asignatura %in% c("CIENCIAS NATURALES", "CIENCIAS SOCIALES", "MATEMATICA", "LENGUA"))

p <- ggplot(datos_filtrados, aes(x = asignatura, y = Promedio, fill = asignatura)) +
  geom_boxplot() +
  ggtitle("                      Comparación de Promedios en materias de tercer grado") +  
  labs(
    x = "Materias",
    y = "Promedios"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )
print(p)

In [ ]:
# Histograma de promedios en matemática
notas_mat_tercero <- filter(Notas_Victoria_3Grado,asignatura == "MATEMATICA")$Promedio
histograma_mat <- fdt(notas_mat_tercero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_mat,
     type= "fh", 
     col= "#FF9932",
     main= "Histograma de los promedios de tercer grado
     en la materia Matemática")

# Medidas de distribución
coef_sim = skewness(notas_mat_tercero, na.rm=TRUE)
kurtosis = kurtosis(notas_mat_tercero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_mat_tercero)

In [ ]:
# Histograma de promedios en lengua
notas_len_tercero <- filter(Notas_Victoria_3Grado,asignatura == "LENGUA")$Promedio
histograma_len <- fdt(notas_len_tercero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_len,
     type= "fh", 
     col= "#FF86FF",
     main= "Histograma de los promedios de tercer grado
     en la materia Lengua")

# Medidas de distribución
coef_sim = skewness(notas_len_tercero, na.rm=TRUE)
kurtosis = kurtosis(notas_len_tercero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_len_tercero)

In [ ]:
# Histograma de promedios en ciencias naturales
notas_nat_tercero <- filter(Notas_Victoria_3Grado,asignatura == "CIENCIAS NATURALES")$Promedio
histograma_nat <- fdt(notas_nat_tercero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_nat,
     type= "fh", 
     col= "#00BB00",
     main= "Histograma de los promedios de tercer grado
     en la materia Ciencias Naturales")

# Medidas de distribución
coef_sim = skewness(notas_nat_tercero, na.rm=TRUE)
kurtosis = kurtosis(notas_nat_tercero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_nat_tercero)

In [ ]:
# Histograma de promedios en ciencias sociales
notas_soc_tercero <- filter(Notas_Victoria_3Grado,asignatura == "CIENCIAS SOCIALES")$Promedio
histograma_soc <- fdt(notas_soc_tercero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_soc,
     type= "fh", 
     col= "#3D87FF",
     main= "Histograma de los promedios de tercer grado
     en la materia Ciencias Sociales")

# Medidas de distribución
coef_sim = skewness(notas_soc_tercero, na.rm=TRUE)
kurtosis = kurtosis(notas_soc_tercero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_soc_tercero)

In [ ]:
# Unimos en un data frame las materias seleccionadas para calcular correlaciones
df <- suppressWarnings(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "CIENCIAS NATURALES"),"Documento","Promedio") %>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "CIENCIAS SOCIALES"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "MATEMATICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "LENGUA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "EDUCACION FISICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "EDUCACION MUSICAL"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "EDUCACION TECNOLOGICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "ARTES VISUALES"),"Documento","Promedio"),by="Documento"))
df <- rename(df,Prom_Nat=Promedio.x, Prom_Soc=Promedio.y, Prom_Mat=Promedio.x.x, Prom_Len=Promedio.y.y, Prom_EdFisica=Promedio.x.x.x, Prom_Mus=Promedio.y.y.y, Prom_Tecno=Promedio.x.x.x.x, Prom_Art=Promedio.y.y.y.y)

In [ ]:
# Gráfico de la matriz de correlación
matriz_correlacion <- cor(df[, -1], method="spearman")
corrplot(matriz_correlacion, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Matriz de correlación", cex.main = 1.5)

In [ ]:
# Gráfico de correlación parcial
variables <- c("Prom_Nat", "Prom_Soc","Prom_Mat", "Prom_Len", "Prom_Mus", "Prom_Tecno", "Prom_Art")
dataM <- df[, variables]
correlacion_parcial <- pcor(dataM, method = "pearson")$estimate
corrplot(correlacion_parcial, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Correlación parcial", cex.main = 1.5)

## Análisis de Cuarto grado

In [ ]:
print("Medidas estadisticas descriptivas de cuarto grado")
Notas_Victoria_4Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>% 
  group_by(asignatura) %>%
  summarise(Conteo = n(),
            Media = round(mean(Promedio, na.rm = TRUE), 2),
            Mediana = round(median(Promedio, na.rm = TRUE), 2),
            Moda = Mode(Promedio),            
            Varianza = round(var(Promedio, na.rm = TRUE), 2),
            Desvio = round(sd(Promedio, na.rm = TRUE), 2),
            Minimo = round(min(Promedio, na.rm = TRUE), 2),
            Maximo = round(max(Promedio, na.rm = TRUE), 2),
            Rango = Maximo - Minimo,
            IQR = round(IQR(Promedio, na.rm = TRUE), 2),
            Q1 = round(quantile(Promedio, 0.25, na.rm = TRUE), 2),
            Q3 = round(quantile(Promedio, 0.75, na.rm = TRUE), 2),
            Sup. = round((Q3 + 1.5 * IQR), 2),
            Inf. = round((Q1 - 1.5 * IQR), 2),
            CoefVar = round(sd(Promedio, na.rm = TRUE) / mean(Promedio, na.rm = TRUE) * 100, 2))

In [ ]:
# Gráfico de boxplot comparativo
datos_filtrados <- Notas_Victoria_4Grado %>%
  filter(asignatura %in% c("CIENCIAS NATURALES", "CIENCIAS SOCIALES", "MATEMATICA", "LENGUA"))

p <- ggplot(datos_filtrados, aes(x = asignatura, y = Promedio, fill = asignatura)) +
  geom_boxplot() +
  ggtitle("                      Comparación de Promedios en materias de cuarto grado") +  
  labs(
    x = "Materias",
    y = "Promedios"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )
print(p)

In [ ]:
# Histograma de promedios en matemática
notas_mat_cuarto <- filter(Notas_Victoria_4Grado,asignatura == "MATEMATICA")$Promedio
histograma_mat <- fdt(notas_mat_cuarto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_mat,
     type= "fh", 
     col= "#FF9932",
     main= "Histograma de los promedios de cuarto grado
     en la materia Matemática")

# Medidas de distribución
coef_sim = skewness(notas_mat_cuarto, na.rm=TRUE)
kurtosis = kurtosis(notas_mat_cuarto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_mat_cuarto)

In [ ]:
# Histograma de promedios en lengua
notas_len_cuarto <- filter(Notas_Victoria_4Grado,asignatura == "LENGUA")$Promedio
histograma_len <- fdt(notas_len_cuarto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_len,
     type= "fh", 
     col= "#FF86FF",
     main= "Histograma de los promedios de cuarto grado
     en la materia Lengua")

# Medidas de distribución
coef_sim = skewness(notas_len_cuarto, na.rm=TRUE)
kurtosis = kurtosis(notas_len_cuarto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_len_cuarto)

In [ ]:
# Histograma de promedios en ciencias naturales
notas_nat_cuarto <- filter(Notas_Victoria_4Grado,asignatura == "CIENCIAS NATURALES")$Promedio
histograma_nat <- fdt(notas_nat_cuarto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_nat,
     type= "fh", 
     col= "#00BB00",
     main= "Histograma de los promedios de cuarto grado
     en la materia Ciencias Naturales")

# Medidas de distribución
coef_sim = skewness(notas_nat_cuarto, na.rm=TRUE)
kurtosis = kurtosis(notas_nat_cuarto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_nat_cuarto)

In [ ]:
# Histograma de promedios en ciencias sociales
notas_soc_cuarto <- filter(Notas_Victoria_4Grado,asignatura == "CIENCIAS SOCIALES")$Promedio
histograma_soc <- fdt(notas_soc_cuarto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_soc,
     type= "fh", 
     col= "#3D87FF",
     main= "Histograma de los promedios de cuarto grado
     en la materia Ciencias Sociales")

# Medidas de distribución
coef_sim = skewness(notas_soc_cuarto, na.rm=TRUE)
kurtosis = kurtosis(notas_soc_cuarto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_soc_cuarto)

In [ ]:
# Unimos en un data frame las materias seleccionadas para calcular correlaciones
df <- suppressWarnings(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "CIENCIAS NATURALES"),"Documento","Promedio") %>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "CIENCIAS SOCIALES"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "MATEMATICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "LENGUA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "EDUCACION FISICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "EDUCACION MUSICAL"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "EDUCACION TECNOLOGICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "ARTES VISUALES"),"Documento","Promedio"),by="Documento"))
df <- rename(df,Prom_Nat=Promedio.x, Prom_Soc=Promedio.y, Prom_Mat=Promedio.x.x, Prom_Len=Promedio.y.y, Prom_EdFisica=Promedio.x.x.x, Prom_Mus=Promedio.y.y.y, Prom_Tecno=Promedio.x.x.x.x, Prom_Art=Promedio.y.y.y.y)

In [ ]:
# Gráfico de la matriz de correlación
matriz_correlacion <- cor(df[, -1], method="spearman")
corrplot(matriz_correlacion, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Matriz de correlación", cex.main = 1.5)

# Gráfico de correlación parcial
correlacion_parcial <- pcor(df[, -1], method = "pearson")$estimate
corrplot(correlacion_parcial, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Correlación parcial", cex.main = 1.5)

## Análisis de Quinto grado

In [ ]:
print("Medidas estadisticas descriptivas de quinto grado")
Notas_Victoria_5Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>% 
  group_by(asignatura) %>%
  summarise(Conteo = n(),
            Media = round(mean(Promedio, na.rm = TRUE), 2),
            Mediana = round(median(Promedio, na.rm = TRUE), 2),
            Moda = Mode(Promedio),            
            Varianza = round(var(Promedio, na.rm = TRUE), 2),
            Desvio = round(sd(Promedio, na.rm = TRUE), 2),
            Minimo = round(min(Promedio, na.rm = TRUE), 2),
            Maximo = round(max(Promedio, na.rm = TRUE), 2),
            Rango = Maximo - Minimo,
            IQR = round(IQR(Promedio, na.rm = TRUE), 2),
            Q1 = round(quantile(Promedio, 0.25, na.rm = TRUE), 2),
            Q3 = round(quantile(Promedio, 0.75, na.rm = TRUE), 2),
            Sup. = round((Q3 + 1.5 * IQR), 2),
            Inf. = round((Q1 - 1.5 * IQR), 2),
            CoefVar = round(sd(Promedio, na.rm = TRUE) / mean(Promedio, na.rm = TRUE) * 100, 2))

In [ ]:
# Gráfico de boxplot comparativo
datos_filtrados <- Notas_Victoria_5Grado %>%
  filter(asignatura %in% c("CIENCIAS NATURALES", "CIENCIAS SOCIALES", "MATEMATICA", "LENGUA"))

p <- ggplot(datos_filtrados, aes(x = asignatura, y = Promedio, fill = asignatura)) +
  geom_boxplot() +
  ggtitle("                      Comparación de Promedios en materias de quinto grado") +  
  labs(
    x = "Materias",
    y = "Promedios"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )
print(p)

In [ ]:
# Histograma de promedios en matemática
notas_mat_quinto <- filter(Notas_Victoria_5Grado,asignatura == "MATEMATICA")$Promedio
histograma_mat <- fdt(notas_mat_quinto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_mat,
     type= "fh", 
     col= "#FF9932",
     main= "Histograma de los promedios de quinto grado
     en la materia Matemática")

# Medidas de distribución
coef_sim = skewness(notas_mat_quinto, na.rm=TRUE)
kurtosis = kurtosis(notas_mat_quinto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_mat_quinto)

In [ ]:
# Histograma de promedios en lengua
notas_len_quinto <- filter(Notas_Victoria_5Grado,asignatura == "LENGUA")$Promedio
histograma_len <- fdt(notas_len_quinto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_len,
     type= "fh", 
     col= "#FF86FF",
     main= "Histograma de los promedios de quinto grado
     en la materia Lengua")

# Medidas de distribución
coef_sim = skewness(notas_len_quinto, na.rm=TRUE)
kurtosis = kurtosis(notas_len_quinto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_len_quinto)

In [ ]:
# Histograma de promedios en ciencias naturales
notas_nat_quinto <- filter(Notas_Victoria_5Grado,asignatura == "CIENCIAS NATURALES")$Promedio
histograma_nat <- fdt(notas_nat_quinto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_nat,
     type= "fh", 
     col= "#00BB00",
     main= "Histograma de los promedios de quinto grado
     en la materia Ciencias Naturales")

# Medidas de distribución
coef_sim = skewness(notas_nat_quinto, na.rm=TRUE)
kurtosis = kurtosis(notas_nat_quinto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_nat_quinto)

In [ ]:
# Histograma de promedios en ciencias sociales
notas_soc_quinto <- filter(Notas_Victoria_5Grado,asignatura == "CIENCIAS SOCIALES")$Promedio
histograma_soc <- fdt(notas_soc_quinto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_soc,
     type= "fh", 
     col= "#3D87FF",
     main= "Histograma de los promedios de quinto grado
     en la materia Ciencias Sociales")

# Medidas de distribución
coef_sim = skewness(notas_soc_quinto, na.rm=TRUE)
kurtosis = kurtosis(notas_soc_quinto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_soc_quinto)

In [ ]:
# Unimos en un data frame las materias seleccionadas para calcular correlaciones
df <- suppressWarnings(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "CIENCIAS NATURALES"),"Documento","Promedio") %>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "CIENCIAS SOCIALES"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "MATEMATICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "LENGUA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "EDUCACION FISICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "EDUCACION MUSICAL"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "EDUCACION TECNOLOGICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "ARTES VISUALES"),"Documento","Promedio"),by="Documento"))
df <- rename(df,Prom_Nat=Promedio.x, Prom_Soc=Promedio.y, Prom_Mat=Promedio.x.x, Prom_Len=Promedio.y.y, Prom_EdFisica=Promedio.x.x.x, Prom_Mus=Promedio.y.y.y, Prom_Tecno=Promedio.x.x.x.x, Prom_Art=Promedio.y.y.y.y)

In [ ]:
# Gráfico de la matriz de correlación
matriz_correlacion <- cor(df[, -1], method="spearman")
corrplot(matriz_correlacion, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Matriz de correlación", cex.main = 1.5)

# Gráfico de correlación parcial
variables <- c("Prom_Nat", "Prom_Soc", "Prom_Mat", "Prom_Len", "Prom_Tecno")
dataM <- df[, variables]
correlacion_parcial <- pcor(dataM, method = "pearson")$estimate
corrplot(correlacion_parcial, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Correlación parcial", cex.main = 1.5)

## Análisis de Sexto grado

In [ ]:
print("Medidas estadisticas descriptivas de sexto grado")
Notas_Victoria_6Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>% 
  group_by(asignatura) %>%
  summarise(Conteo = n(),
            Media = round(mean(Promedio, na.rm = TRUE), 2),
            Mediana = round(median(Promedio, na.rm = TRUE), 2),
            Moda = Mode(Promedio),            
            Varianza = round(var(Promedio, na.rm = TRUE), 2),
            Desvio = round(sd(Promedio, na.rm = TRUE), 2),
            Minimo = round(min(Promedio, na.rm = TRUE), 2),
            Maximo = round(max(Promedio, na.rm = TRUE), 2),
            Rango = Maximo - Minimo,
            IQR = round(IQR(Promedio, na.rm = TRUE), 2),
            Q1 = round(quantile(Promedio, 0.25, na.rm = TRUE), 2),
            Q3 = round(quantile(Promedio, 0.75, na.rm = TRUE), 2),
            Sup. = round((Q3 + 1.5 * IQR), 2),
            Inf. = round((Q1 - 1.5 * IQR), 2),
            CoefVar = round(sd(Promedio, na.rm = TRUE) / mean(Promedio, na.rm = TRUE) * 100, 2))

In [ ]:
# Gráfico de boxplot comparativo
datos_filtrados <- Notas_Victoria_6Grado %>%
  filter(asignatura %in% c("CIENCIAS NATURALES", "CIENCIAS SOCIALES", "MATEMATICA", "LENGUA"))

p <- ggplot(datos_filtrados, aes(x = asignatura, y = Promedio, fill = asignatura)) +
  geom_boxplot() +
  ggtitle("                      Comparación de Promedios en materias de sexto grado") +  
  labs(
    x = "Materias",
    y = "Promedios"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )
print(p)

In [ ]:
# Histograma de promedios en matemática
notas_mat_sexto <- filter(Notas_Victoria_6Grado,asignatura == "MATEMATICA")$Promedio
histograma_mat <- fdt(notas_mat_sexto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_mat,
     type= "fh", 
     col= "#FF9932",
     main= "Histograma de los promedios de sexto grado
     en la materia Matemática")

# Medidas de distribución
coef_sim = skewness(notas_mat_sexto, na.rm=TRUE)
kurtosis = kurtosis(notas_mat_sexto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_mat_sexto)

In [ ]:
# Histograma de promedios en lengua
notas_len_sexto <- filter(Notas_Victoria_6Grado,asignatura == "LENGUA")$Promedio
histograma_len <- fdt(notas_len_sexto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_len,
     type= "fh", 
     col= "#FF86FF",
     main= "Histograma de los promedios de sexto grado
     en la materia Lengua")

# Medidas de distribución
coef_sim = skewness(notas_len_sexto, na.rm=TRUE)
kurtosis = kurtosis(notas_len_sexto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_len_sexto)

In [ ]:
# Histograma de promedios en ciencias naturales
notas_nat_sexto <- filter(Notas_Victoria_6Grado,asignatura == "CIENCIAS NATURALES")$Promedio
histograma_nat <- fdt(notas_nat_sexto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_nat,
     type= "fh", 
     col= "#00BB00",
     main= "Histograma de los promedios de sexto grado
     en la materia Ciencias Naturales")

# Medidas de distribución
coef_sim = skewness(notas_nat_sexto, na.rm=TRUE)
kurtosis = kurtosis(notas_nat_sexto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_nat_sexto)

In [ ]:
# Histograma de promedios en ciencias sociales
notas_soc_sexto <- filter(Notas_Victoria_6Grado,asignatura == "CIENCIAS SOCIALES")$Promedio
histograma_soc <- fdt(notas_soc_sexto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_soc,
     type= "fh", 
     col= "#3D87FF",
     main= "Histograma de los promedios de sexto grado
     en la materia Ciencias Sociales")

# Medidas de distribución
coef_sim = skewness(notas_soc_sexto, na.rm=TRUE)
kurtosis = kurtosis(notas_soc_sexto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_soc_sexto)

In [ ]:
# Unimos en un data frame las materias seleccionadas para calcular correlaciones
df <- suppressWarnings(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "CIENCIAS NATURALES"),"Documento","Promedio") %>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "CIENCIAS SOCIALES"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "MATEMATICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "LENGUA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "EDUCACION FISICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "EDUCACION MUSICAL"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "EDUCACION TECNOLOGICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "ARTES VISUALES"),"Documento","Promedio"),by="Documento"))
df <- rename(df,Prom_Nat=Promedio.x, Prom_Soc=Promedio.y, Prom_Mat=Promedio.x.x, Prom_Len=Promedio.y.y, Prom_EdFisica=Promedio.x.x.x, Prom_Mus=Promedio.y.y.y, Prom_Tecno=Promedio.x.x.x.x, Prom_Art=Promedio.y.y.y.y)

In [ ]:
# Gráfico de la matriz de correlación
matriz_correlacion <- cor(df[, -1], method="spearman")
corrplot(matriz_correlacion, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Matriz de correlación", cex.main = 1.5)

# Gráfico de correlación parcial
variables <- c("Prom_Nat", "Prom_Soc", "Prom_Mat", "Prom_Len")
dataM <- df[, variables]
correlacion_parcial <- pcor(dataM, method = "pearson")$estimate
corrplot(correlacion_parcial, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Correlación parcial", cex.main = 1.5)

## Anexo

In [ ]:
# Si queremos correr esta celda, restaurar el kernel y no correr la segunda celda de librerias, corrompe los datos
library(tidyr)
datos_trimestres <- Notas_Victoria_1Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>%
  select(NotasPrimerT, NotasSegundoT, NotasTercerT)  # Selecciona las columnas de notas por trimestre

# Convertir los datos en un formato largo (tidy)
datos_trimestres_long <- pivot_longer(datos_trimestres, 
                                      cols = everything(), 
                                      names_to = "Trimestre", 
                                      values_to = "Notas")

# Crear un boxplot comparativo por trimestre
p <- ggplot(datos_trimestres_long, aes(x = Trimestre, y = Notas, fill = Trimestre)) +
  geom_boxplot() +
  labs(
    title = "Comparación de Notas por Trimestre",
    subtitle = "Fuente de elaboración propia",
    x = "Trimestre",
    y = "Notas"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 0, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )

# Mostrar el gráfico
print(p)